In [6]:
import pprint as pp
import json

In [7]:
# starting code that loads data, run this cell, but no need to understand it in depth
# this function just loads the data from files
def load_json_file_named(file_name):
    try: 
        loaded_data = []
        file_location = f"{file_name}"
        with open(file_location, 'r') as file: # or f"data/{file_name}" depending on your files
            loaded_data =  json.load(file)
    except OSError as e:
        print(f"Error. Does the file exist in this folder? {file_location}\n\n {e}")
    return loaded_data

In [8]:
boards_info = load_json_file_named('nhs_scotland_boards.json')
covid_days = load_json_file_named('covid_records_scotland.json')
cancer_waiting_times = load_json_file_named('cancer_waiting_times.json')
boards_code_lookup = load_json_file_named('boards_code_lookup.json')

In [105]:
def running_average(numbers):
    new_list = []
    for i in range(len(numbers)):
        new_list.append(numbers[i] + sum(numbers[:i]))
    return new_list

In [106]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

def plot_time_series(dic,key):
    new_positive=[]
    new_deaths=[]
    first_infections=[]
    reinfections=[]
    date=[]
    for i in dic:
        if i['locations'][key]=={}:
            continue
        else:
            new_positive.append(i['locations'][key]['new_positive'])
            new_deaths.append(i['locations'][key]['new_deaths'])
            first_infections.append(i['locations'][key]['first_infections'])
            reinfections.append(i['locations'][key]['reinfections'])
            date.append(datetime.strptime(str(i['date']),"%Y%m%d"))
    new_positive=running_average(new_positive)
    new_deaths=running_average(new_deaths)
    first_infections=running_average(first_infections)
    reinfections=running_average(reinfections)
    fig = make_subplots(rows=4, cols=1)
    fig.add_trace(go.Scatter(x=date, y=new_positive,name="New Positive"),
    row=1, col=1)
    fig.add_trace(go.Scatter(x=date, y=new_deaths,name="New Deaths"),
    row=2, col=1)
    fig.add_trace(go.Scatter(x=date, y=first_infections,name="First Infections"),
    row=3, col=1)
    fig.add_trace(go.Scatter(x=date, y=reinfections,name="Reinfections"),
    row=4, col=1)
    fig.update_layout(height=800,width=1000,title_text=str(key)+" : Time series graphs")
    fig.show()

In [107]:
for i in covid_days[0]['locations'].keys():
    if i=='date':
        continue
    else:
        plot_time_series(covid_days,i)